In [2]:
from datetime import datetime
import random
import os
import getpass
from huggingface_hub import InferenceClient
from langchain_core.prompts import PromptTemplate
from secret_key import HUGGINGFACEHUB_API_TOKEN



#function to generate a unique work order number
def generate_work_order_number():
    today = datetime.now()
    random_digits = random.randint(1000, 9999)
    return f"WO-{today.strftime('%Y%m%d')}-{random_digits}"

# Function to get user input
def get_user_input():
    company_name = input("Enter your company name: ")
    contractor_name = input("Enter the contractor name: ")
    project_name = input("Enter the project name: ")
    location = input("Enter the project location: ")
    
    work_description = input("Enter a description of the work to be performed: ")
    estimated_duration = input("Enter the estimated duration (in days): ")
    
    return {
        "company_name": company_name,
        "contractor_name": contractor_name,
        "project_name": project_name,
        "location": location,
        "work_description": work_description,
        "estimated_duration": estimated_duration,
        "work_order_number": generate_work_order_number(),
        "date_issued": datetime.now().strftime("%Y-%m-%d")
    }



# function for api_key
def get_api_key():
    api_key = HUGGINGFACEHUB_API_TOKEN
    return api_key



# Function to generate text using the Hugging Face Inference API
def generate_text_with_hf(prompt, api_key):
    client = InferenceClient(token=api_key)
    
    # Using text generation specific method
    result = client.text_generation(
        prompt=prompt,
        model="mistralai/Mistral-7B-Instruct-v0.2",
        temperature=0.7,
        max_new_tokens=2048,
        do_sample=True
    )
    
    return result

# Format the template with input data
def format_prompt(template, data):
    prompt = PromptTemplate(
        input_variables=["company_name", "contractor_name", "project_name", "location", 
                        "work_description", "estimated_duration", "work_order_number", "date_issued"],
        template=template
    )
    return prompt.format(**data)

#Main function to run the work order generator
def generate_work_order():
    print("=== Construction Work Order Generator ===")
    print("Powered by Hugging Face Inference API")
    
    # Get the API key securely
    api_key = get_api_key()
    
    if not api_key:
        print("Error: No API key provided. Cannot continue.")
        return
    
    # Get user inputs
    input_data = get_user_input()
    
    # Create a prompt template
    template = """
    Create a formal construction work order with the following information:
    
    Work Order Number: {work_order_number}
    Date Issued: {date_issued}
    
    Company Information:
    Company Name: {company_name}
    
    Contractor Information:
    Contractor Name: {contractor_name}
    
    Project Details:
    Project Name: {project_name}
    Location: {location}
    
    Work Description:
    {work_description}
    
    Estimated Duration: {estimated_duration} days
    
    Include sections for:
    1. Scope of Work (elaborate on the work description in a professional manner)
    2. Materials Required (suggest likely materials based on the work description)
    3. Safety Requirements (standard construction safety protocols)
    4. Quality Control Measures
    5. Approval section with signature lines for both company representative and contractor
    
    Format this as a professional work order document.
    """
    
    try:
        # Format the prompt with the input data
        formatted_prompt = format_prompt(template, input_data)
        
        print("\nGenerating work order...\n")
        
        # Generate the work order directly using the Hugging Face Inference API
        work_order = generate_text_with_hf(formatted_prompt, api_key)
        
        # Save to file
        filename = f"Work_Order_{input_data['work_order_number']}.txt"
        with open(filename, "w") as f:
            f.write(work_order)
        
        print(f"Work order has been generated and saved as '{filename}'")
        print("\n" + "="*10 + "\n")
        print(work_order)
    
    except Exception as e:
        print(f"Error generating work order: {str(e)}")


if __name__ == "__main__":
    generate_work_order()

=== Construction Work Order Generator ===
Powered by Hugging Face Inference API


Enter your company name:  ahjin construction
Enter the contractor name:  harsh singh
Enter the project name:  mca bulding construction
Enter the project location:  banglore
Enter a description of the work to be performed:  constructing a mca building for pes college
Enter the estimated duration (in days):  300



Generating work order...

Work order has been generated and saved as 'Work_Order_WO-20250225-6739.txt'



    Work Order Number: WO-20250225-6739
    Date Issued: 25th February, 2025

    Ahjin Construction
    1234, 5th Avenue
    Bangalore, 560001

    Work Order

    To: Harsh Singh
    Contractor
    7890, 1st Street
    Bangalore, 560002

    Subject: MCA Building Construction Work Order

    Scope of Work:
    
    Ahjin Construction hereby orders Harsh Singh, Contractor, to construct a new MCA (Master of Computer Applications) building for PES College, located at [insert location], Bangalore, India. The work includes but is not limited to the following tasks:

    1. Preparing and clearing the site for construction, including the removal of debris and vegetation.
    2. Excavating and preparing the foundation.
    3. Building the structural framework, including columns, beams, and girders.
    4. Installing floors, walls, and roofing.
    5. Installing doors, windows, and other